In [ ]:
!pip install addict
!pip install yapf
!pip install timm
!pip install pycocotools matplotlib tqdm
from collections.abc import Iterable
!pip install tqdm matplotlib
import sys
import torch
from pathlib import Path
import torchvision.transforms as T
import importlib.util
from util.slconfig import SLConfig
import os
from tqdm import tqdm
import matplotlib.pyplot as plt
from PIL import Image
import json
import numpy as np


In [ ]:
# Change to the correct directory
%cd "/DINO/models/dino/ops/"

# Run the setup script to compile the CUDA extensions
!python setup.py build install


In [ ]:

sys.path.append('/DINO')  # Correct path to your DINO repo


# Manually import the modified coco.py from the specified path
spec = importlib.util.spec_from_file_location("coco", "/DINO/datasets/coco.py")   # Correct path to your coco.py file

coco = importlib.util.module_from_spec(spec)
spec.loader.exec_module(coco)

# Argument class to simulate the input to the build function
class Args:
    def __init__(self):
        self.coco_path = "  place holder "  # Path to your dataset folder
        self.modelname = 'dino' 
        self.fix_size = False  
        self.masks = False  


# Custom collate function to handle variable image sizes
def collate_fn(batch):
    images, targets = zip(*batch)
    # Pad images to the same size
    images = [T.functional.resize(image, (800, 1333)) for image in images] 
    images = torch.stack(images, dim=0) 
    return images, targets


# Initialize the dataset using the build function
args = Args()
image_set = "train"  # You can switch between "train" and "val" to test both
dataset = coco.build(image_set, args)
data_loader = torch.utils.data.DataLoader(dataset, batch_size=2, shuffle=True, collate_fn=collate_fn)


for images, targets in data_loader:
    print(f"Loaded {len(images)} images and {len(targets)} annotations")
    print(f"Image sizes: {[img.size() for img in images]}")
    print(f"Annotations: {targets}")
    break  


In [ ]:
!ls "DINO/models/dino/ops/build/lib.linux-x86_64-3.*/MultiScaleDeformableAttention*"
# Navigate to the ops directory
%cd "DINO/models/dino/ops/"

# Capture the log of the build process
!python setup.py build install > build_log.txt 2>&1

# Display the last 50 lines of the build log
!tail -n 50 build_log.txt

!pip list



In [5]:

# Adding paths to the compiled MultiScaleDeformableAttention module
sys.path.append('/usr/local/lib/python3.10/dist-packages/MultiScaleDeformableAttention-1.0-py3.10-linux-x86_64.egg')
sys.path.append('DINO/models/dino/ops/build/lib.linux-x86_64-cpython-310/')

# Try importing the module again
import MultiScaleDeformableAttention


In [ ]:

for key, value in vars(args).items():
    print(f"{key}: {value}")


In [ ]:


def check_config(config_file):
    args = SLConfig.fromfile(config_file)

    required_params = [
        'dataset_file', 'coco_path', 'num_classes', 'lr', 'batch_size',
        'weight_decay', 'epochs', 'lr_drop', 'modelname', 'backbone',
        'num_queries', 'aux_loss', 'set_cost_class', 'set_cost_bbox',
        'set_cost_giou', 'mask_loss_coef', 'dice_loss_coef', 'bbox_loss_coef',
        'giou_loss_coef'
    ]

    missing_params = []
    for param in required_params:
        if not hasattr(args, param):
            missing_params.append(param)

    if missing_params:
        print(f"Missing parameters in {config_file}:")
        for param in missing_params:
            print(f"- {param}")
    else:
        print(f"All required parameters are present in {config_file}")

    # Check if coco_path exists
    if hasattr(args, 'coco_path'):
        if not os.path.exists(args.coco_path):
            print(f"Warning: coco_path '{args.coco_path}' does not exist")
        else:
            print(f"coco_path '{args.coco_path}' exists")

    # Print all parameters for reference
    print("\nAll parameters:")
    for key, value in vars(args).items():
        print(f"{key}: {value}")

if __name__ == "__main__":
    config_file = 'DINO/config/DINO/DINO_4scale.py'
    check_config(config_file)

In [ ]:


# Update the __init__.py file
init_content = """
from .dino.dino import build_dino

print("Debug: __init__.py loaded")

def build_model(args):
    print("Debug: build_model function called from __init__.py")
    return build_dino(args)

print("Debug: build_model function defined in __init__.py")
"""

init_path = 'DINO/models/__init__.py'
with open(init_path, 'w') as f:
    f.write(init_content)

print(f"Updated {init_path}")


import importlib
import models
importlib.reload(models)


sys.path.append('DINO')

print("Debug: Current working directory:", os.getcwd())
print("Debug: Content of DINO directory:")
print(os.listdir('DINO'))
print("Debug: Content of models directory:")
print(os.listdir('DINO/models'))

print("Debug: Before imports")
from util.slconfig import SLConfig
print("Debug: After SLConfig import")
from models import build_model
print("Debug: After build_model import")
from models.dino.dino import build_dino
print("Debug: After build_dino import")

def test_build_model():
    print("Testing DINO model build...")

    # Load configuration
    config_file = 'DINO/config/DINO/DINO_4scale.py'
    args = SLConfig.fromfile(config_file)

    # Add necessary arguments if they're not in the config file
    if not hasattr(args, 'num_classes'):
        args.num_classes = 91
    if not hasattr(args, 'num_queries'):
        args.num_queries = 900
    if not hasattr(args, 'aux_loss'):
        args.aux_loss = True
    if not hasattr(args, 'with_box_refine'):
        args.with_box_refine = False
    if not hasattr(args, 'two_stage'):
        args.two_stage = False
    if not hasattr(args, 'device'):
        args.device = 'cuda'

    print("\nChecking build_dino import:")
    print(f"build_dino in models.dino.dino: {hasattr(build_dino, '__call__')}")

    try:
        print("\nTrying to build model using build_model:")
        model, criterion, postprocessors = build_model(args)
        print("Model built successfully using build_model!")
        print(f"Model type: {type(model)}")
        print(f"Criterion type: {type(criterion)}")
        print(f"Postprocessors type: {type(postprocessors)}")
    except Exception as e:
        print(f"An error occurred while building the model using build_model: {str(e)}")
        import traceback
        traceback.print_exc()

    try:
        print("\nTrying to build model using build_dino directly:")
        model, criterion, postprocessors = build_dino(args)
        print("Model built successfully using build_dino directly!")
        print(f"Model type: {type(model)}")
        print(f"Criterion type: {type(criterion)}")
        print(f"Postprocessors type: {type(postprocessors)}")
    except Exception as e:
        print(f"An error occurred while building the model using build_dino directly: {str(e)}")
        import traceback
        traceback.print_exc()

if __name__ == "__main__":
    test_build_model()

In [ ]:

!pip install pycocotools

import sys
sys.path.append('DINO')

from util.slconfig import SLConfig
from datasets import build_dataset
from torch.utils.data import DataLoader

def build_and_check_dataset():
    config_file = 'DINO/config/DINO/DINO_4scale.py'
    args = SLConfig.fromfile(config_file)

    try:
        dataset_val = build_dataset(image_set='val', args=args)
        data_loader_val = DataLoader(dataset_val, batch_size=1, shuffle=False, num_workers=args.num_workers)


        print(f"Dataset length: {len(dataset_val)}")
        print(f"Dataset type: {type(dataset_val)}")

        # Check the first item in the dataset
        first_item = next(iter(data_loader_val))
        print("\nFirst item in the dataset:")
        print(f"Images shape: {first_item[0].shape}")
        print(f"Targets type: {type(first_item[1])}")
        print("Targets content:")
        print(first_item[1])

    except Exception as e:
        print(f"An error occurred: {str(e)}")


build_and_check_dataset()

In [ ]:


# Add DINO directory to sys.path
sys.path.append('DINO')

from util.slconfig import SLConfig
from models import build_model
from datasets import build_dataset
from util.misc import nested_tensor_from_tensor_list
from util.box_ops import box_cxcywh_to_xyxy
from pycocotools.coco import COCO
from pycocotools.cocoeval import COCOeval

def load_model(args):
    model, _, postprocessors = build_model(args)
    checkpoint = torch.load(args.resume, map_location='cpu')
    model.load_state_dict(checkpoint['model'])
    model.eval()
    return model, postprocessors

def process_image(image_path, model, postprocessors, device):
    image = Image.open(image_path).convert("RGB")
    w, h = image.size
    img = torch.from_numpy(np.array(image)).permute(2, 0, 1).float()
    img = nested_tensor_from_tensor_list([img]).to(device)

    with torch.no_grad():
        outputs = model(img)

    results = postprocessors['bbox'](outputs, torch.tensor([[w, h]]).to(device))[0]
    return results

def visualize_detections(image, boxes, labels, scores, output_path):
    plt.figure(figsize=(12, 8))
    plt.imshow(image)
    ax = plt.gca()

    for box, label, score in zip(boxes, labels, scores):
        x1, y1, x2, y2 = box
        rect = plt.Rectangle((x1, y1), x2 - x1, y2 - y1, fill=False, edgecolor='red', linewidth=2)
        ax.add_patch(rect)
        plt.text(x1, y1, f'{label}: {score:.2f}', bbox=dict(facecolor='white', alpha=0.8))

    plt.axis('off')
    plt.savefig(output_path, bbox_inches='tight', pad_inches=0)
    plt.close()

def main():
    # Load configuration
    config_file = 'DINO/config/DINO/DINO_4scale.py'
    args = SLConfig.fromfile(config_file)
    args.device = 'cuda'
    args.resume = 'DINO-4scale model/checkpoint0033_4scale.pth'

    # Create output directory
    output_dir = 'output path'
    os.makedirs(output_dir, exist_ok=True)

    # Load model
    model, postprocessors = load_model(args)
    model.to(args.device)

    # Load validation dataset
    dataset_val = build_dataset(image_set='val', args=args)

    # Initialize COCO API
    ann_file = os.path.join(args.coco_path, 'val', 'val.json')
    coco_gt = COCO(ann_file)

    results = []

    for img_id in tqdm(dataset_val.ids):
        # Get image path
        img_info = dataset_val.coco.loadImgs(img_id)[0]
        img_path = os.path.join(dataset_val.root, img_info['file_name'])

        # Process image
        outputs = process_image(img_path, model, postprocessors, args.device)

        # Convert to COCO format
        boxes = outputs['boxes'].tolist()
        scores = outputs['scores'].tolist()
        labels = outputs['labels'].tolist()

        for box, score, label in zip(boxes, scores, labels):
            xmin, ymin, xmax, ymax = box
            width = xmax - xmin
            height = ymax - ymin
            results.append({
                'image_id': img_id,
                'category_id': label,
                'bbox': [xmin, ymin, width, height],
                'score': score
            })

        # Visualize detections
        image = Image.open(img_path).convert("RGB")
        output_path = os.path.join(output_dir, f'detection_{img_id}.png')
        visualize_detections(image, boxes, labels, scores, output_path)

    # Save results for evaluation
    result_path = os.path.join(output_dir, 'detections.json')
    with open(result_path, 'w') as f:
        json.dump(results, f)

    # Evaluate using COCO API
    coco_dt = coco_gt.loadRes(result_path)
    coco_eval = COCOeval(coco_gt, coco_dt, 'bbox')
    coco_eval.evaluate()
    coco_eval.accumulate()
    coco_eval.summarize()

    # Save AP and AR values
    metrics_path = os.path.join(output_dir, 'metrics.txt')
    with open(metrics_path, 'w') as f:
        f.write(f"AP @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = {coco_eval.stats[0]:.3f}\n")
        f.write(f"AP @[ IoU=0.50      | area=   all | maxDets=100 ] = {coco_eval.stats[1]:.3f}\n")
        f.write(f"AP @[ IoU=0.75      | area=   all | maxDets=100 ] = {coco_eval.stats[2]:.3f}\n")
        f.write(f"AP @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = {coco_eval.stats[3]:.3f}\n")
        f.write(f"AP @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = {coco_eval.stats[4]:.3f}\n")
        f.write(f"AP @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = {coco_eval.stats[5]:.3f}\n")
        f.write(f"AR @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = {coco_eval.stats[6]:.3f}\n")
        f.write(f"AR @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = {coco_eval.stats[7]:.3f}\n")
        f.write(f"AR @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = {coco_eval.stats[8]:.3f}\n")
        f.write(f"AR @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = {coco_eval.stats[9]:.3f}\n")
        f.write(f"AR @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = {coco_eval.stats[10]:.3f}\n")
        f.write(f"AR @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = {coco_eval.stats[11]:.3f}\n")



if __name__ == "__main__":
    main()

In [ ]:
import torch
from torch.utils.data import DataLoader
import torchvision.transforms as T
from util.slconfig import SLConfig
from datasets import build_dataset

# Custom collate function to resize tensors directly if they are not PIL Images
def collate_fn(batch):
    images, targets = zip(*batch)

    # Process each image
    processed_images = []
    for image in images:
        if isinstance(image, torch.Tensor):
            # If image is already a tensor, resize using interpolation
            image = torch.nn.functional.interpolate(image.unsqueeze(0), size=(800, 800), mode="bilinear", align_corners=False).squeeze(0)
        else:
            # If image is a PIL Image or ndarray, use torchvision.transforms
            transform = T.Compose([
                T.Resize((800, 800)),  # Resize all images to 800x800
                T.ToTensor()           # Convert images to tensor format
            ])
            image = transform(image)
        processed_images.append(image)

    # Stack the images into a batch (batch_size, channels, height, width)
    images = torch.stack(processed_images, dim=0)

    return images, targets

# Function to build datasets and check data loading
def build_and_check_dataset_with_collate():
    config_file = 'DINO/config/DINO/DINO_4scale.py'
    args = SLConfig.fromfile(config_file)

    try:
        dataset_train = build_dataset(image_set='train', args=args)
        dataset_val = build_dataset(image_set='val', args=args)

        # DataLoader with the custom collate function
        data_loader_train = DataLoader(dataset_train, batch_size=2, shuffle=True, collate_fn=collate_fn)
        data_loader_val = DataLoader(dataset_val, batch_size=2, shuffle=False, collate_fn=collate_fn)

        print(f"Train dataset length: {len(dataset_train)}")
        print(f"Validation dataset length: {len(dataset_val)}")

        # Checking if data is loaded correctly
        first_train_batch = next(iter(data_loader_train))
        first_val_batch = next(iter(data_loader_val))

        print("\nFirst batch of training data:")
        print(f"Images shape: {first_train_batch[0].shape}")
        print(f"Targets content: {first_train_batch[1]}")

        print("\nFirst batch of validation data:")
        print(f"Images shape: {first_val_batch[0].shape}")
        print(f"Targets content: {first_val_batch[1]}")

    except Exception as e:
        print(f"Error occurred: {str(e)}")
        print("Please check your dataset paths and configuration.")

build_and_check_dataset_with_collate()


In [ ]:
import torch
from torch.utils.data import DataLoader
from torchvision import transforms as T
from datasets import build_dataset
from util.slconfig import SLConfig

# Custom collate function to resize images dynamically
def collate_fn(batch):
    images, targets = zip(*batch)

    # Define a transform only for PIL images
    transform = T.Resize((800, 800))

    processed_images = []
    for image in images:
        if isinstance(image, torch.Tensor):
            # If image is already a tensor, resize it using interpolation
            image = torch.nn.functional.interpolate(image.unsqueeze(0), size=(800, 800), mode="bilinear", align_corners=False).squeeze(0)
        else:
            # If image is a PIL Image or ndarray, apply the transform
            image = T.ToTensor()(transform(image))
        processed_images.append(image)

    # Stack the images into a batch (batch_size, channels, height, width)
    images = torch.stack(processed_images, dim=0)

    return images, targets

# Step 2: Verify dataset loading and targets after resizing
def check_dataset_and_targets():
    # Load the configuration and dataset
    config_file = 'DINO/config/DINO/DINO_4scale.py'
    args = SLConfig.fromfile(config_file)

    dataset_train = build_dataset(image_set='train', args=args)

    # Use a DataLoader with the updated collate_fn
    data_loader_train = DataLoader(dataset_train, batch_size=2, shuffle=True, collate_fn=collate_fn)

    # Fetch the first batch of data
    images, targets = next(iter(data_loader_train))

    print(f"Number of images in batch: {len(images)}")
    print(f"Image shape: {images.shape}")
    print(f"Number of targets: {len(targets)}")

    # Check the targets for each image
    for i, target in enumerate(targets):
        print(f"Image {i}:")
        print(f"  Labels: {target.get('labels', 'No labels found')}")
        print(f"  Boxes: {target.get('boxes', 'No boxes found')}")

check_dataset_and_targets()


In [ ]:
import torch
from torch.utils.data import DataLoader
from torchvision import transforms as T
from PIL import Image
from datasets import build_dataset
from util.slconfig import SLConfig

# Custom collate function to gather information about the image sizes
def collate_fn(batch):
    images, targets = zip(*batch)

    # Define the resize transformation
    transform = T.Compose([
        T.Resize((800, 800)),  # Force resize to 800x800
        T.ToTensor()           # Convert to tensor
    ])

    processed_images = []

    # Process each image
    for i, image in enumerate(images):
        if isinstance(image, Image.Image):
            print(f"Image {i}: Original size: {image.size}")
            resized_image = transform(image)
            processed_images.append(resized_image)
            print(f"Image {i}: Resized to: {resized_image.shape}")
        else:
            print(f"Image {i}: Not a PIL image (already a tensor)")
            processed_images.append(image)

    # Attempt to stack the images into a batch and print the result
    try:
        images = torch.stack(processed_images, dim=0)
        print(f"Stacked images shape: {images.shape}")
    except RuntimeError as e:
        print(f"Error when stacking images: {e}")

    return processed_images, targets

# Step 1: Load dataset and gather information about image sizes and targets
def gather_image_info():
    # Load configuration and dataset
    config_file = 'DINO/config/DINO/DINO_4scale.py'
    args = SLConfig.fromfile(config_file)

    dataset_train = build_dataset(image_set='train', args=args)
    data_loader_train = DataLoader(dataset_train, batch_size=2, shuffle=True, collate_fn=collate_fn)

    # Print the sizes of images in the first batch and inspect targets
    for images, targets in data_loader_train:
        print(f"\nNumber of images in batch: {len(images)}")

        # Check target structure
        for i, target in enumerate(targets):
            print(f"\nTarget {i}:")
            if target is None:
                print("Target is None")
            else:
                print(f"  Labels: {target.get('labels', 'No labels found')}")
                print(f"  Boxes: {target.get('boxes', 'No boxes found')}")
                print(f"  Other info: {target.keys()}")

        # Exit after first batch to prevent overwhelming output
        break

# Step 2: Gather the information by running the function
gather_image_info()


In [ ]:
import torch
import torch.nn.functional as F
from torch.utils.data import DataLoader
from datasets import build_dataset
from util.slconfig import SLConfig

# Custom collate function to resize tensor images and stack them
def collate_fn(batch):
    images, targets = zip(*batch)

    # Define the target size for resizing (800x800)
    target_size = (800, 800)

    processed_images = []

    # Process each image
    for i, image in enumerate(images):
        if isinstance(image, torch.Tensor):
            print(f"Image {i}: Original size: {image.shape}")
            # Resize tensor image to (800, 800) using interpolate
            resized_image = F.interpolate(image.unsqueeze(0), size=target_size, mode='bilinear', align_corners=False).squeeze(0)
            processed_images.append(resized_image)
            print(f"Image {i}: Resized to: {resized_image.shape}")
        else:
            print(f"Image {i}: Unexpected data type")
            processed_images.append(image)

    # Stack the images into a batch (batch_size, channels, height, width)
    try:
        images = torch.stack(processed_images, dim=0)
        print(f"Stacked images shape: {images.shape}")
    except RuntimeError as e:
        print(f"Error when stacking images: {e}")

    return images, targets

# Step 1: Load dataset and gather information about image sizes and targets
def gather_image_info():
    # Load configuration and dataset
    config_file = 'DINO/config/DINO/DINO_4scale.py'
    args = SLConfig.fromfile(config_file)

    dataset_train = build_dataset(image_set='train', args=args)
    data_loader_train = DataLoader(dataset_train, batch_size=2, shuffle=True, collate_fn=collate_fn)

    # Print the sizes of images in the first batch and inspect targets
    for images, targets in data_loader_train:
        print(f"\nNumber of images in batch: {len(images)}")

        # Check target structure
        for i, target in enumerate(targets):
            print(f"\nTarget {i}:")
            if target is None:
                print("Target is None")
            else:
                print(f"  Labels: {target.get('labels', 'No labels found')}")
                print(f"  Boxes: {target.get('boxes', 'No boxes found')}")
                print(f"  Other info: {target.keys()}")

        # Exit after first batch to prevent overwhelming output
        break

# Step 2: Gather the information by running the function
gather_image_info()


In [ ]:
import torch
import torch.nn.functional as F
from torch.utils.data import DataLoader
from datasets import build_dataset
from util.slconfig import SLConfig

# Custom collate function to resize tensor images and perform checks
def collate_fn(batch):
    images, targets = zip(*batch)

    # Define the target size for resizing (800x800)
    target_size = (800, 800)

    processed_images = []

    # Process each image
    for image in images:
        if isinstance(image, torch.Tensor):
            resized_image = F.interpolate(image.unsqueeze(0), size=target_size, mode='bilinear', align_corners=False).squeeze(0)
            processed_images.append(resized_image)
        else:
            print("Unexpected image type, expected tensor.")

    images = torch.stack(processed_images, dim=0)

    return images, targets

# Step 1: Load dataset and dataloader to perform pre-checks
def run_pre_checks():
    # Load the configuration and dataset
    config_file = 'DINO/config/DINO/DINO_4scale.py'
    args = SLConfig.fromfile(config_file)

    dataset_train = build_dataset(image_set='train', args=args)
    data_loader_train = DataLoader(dataset_train, batch_size=2, shuffle=True, collate_fn=collate_fn)

    # Counters for NoneType targets and total batches
    none_type_count = 0
    batch_count = 0

    # Iterate over the dataset and perform checks
    for batch_idx, (images, targets) in enumerate(data_loader_train):
        batch_count += 1
        print(f"\nBatch {batch_idx + 1} - Number of images: {len(images)}")

        # Check for NoneType targets and print structure
        for target_idx, target in enumerate(targets):
            if target is None:
                print(f"Target {target_idx} is None.")
                none_type_count += 1
            else:
                print(f"Target {target_idx}: Labels: {target.get('labels', 'No labels found')}, Boxes: {target.get('boxes', 'No boxes found')}")

        # Break after checking a few batches to avoid overwhelming output
        if batch_idx >= 10:
            break

    print(f"\nTotal batches checked: {batch_count}")
    print(f"Total NoneType targets found: {none_type_count}")

# Step 2: Run the pre-checks
run_pre_checks()


In [17]:
import torch

def debug_forward_pass(model, images, targets):
    # Print image and target information
    print(f"Image shapes: {[img.shape for img in images]}")
    print(f"Targets type: {type(targets)}")

    # Log each target
    for idx, target in enumerate(targets):
        if target is None:
            print(f"Target {idx} is None")
        else:
            print(f"Target {idx}: Labels: {target.get('labels', 'No labels')} Boxes: {target.get('boxes', 'No boxes')}")

    # Simulate model forward pass
    if model.dn_number > 0 or targets is not None:
        try:
            # Log the preparation for CDN to understand the flow
            print("Preparing for CDN...")
            input_query_label, input_query_bbox, attn_mask, dn_meta = prepare_for_cdn(
                dn_args=(targets, model.dn_number, model.dn_label_noise_ratio, model.dn_box_noise_scale),
                training=model.training,
                num_queries=model.num_queries,
                num_classes=model.num_classes,
                hidden_dim=model.hidden_dim,
                label_enc=model.label_enc
            )
            print(f"CDN preparation successful. dn_meta: {dn_meta}")
        except Exception as e:
            print(f"Error preparing for CDN: {e}")
            return

    # Check if model can process the images
    try:
        print("Passing images through model...")
        outputs = model(images)
        print("Model forward pass successful.")
    except Exception as e:
        print(f"Error during forward pass: {e}")

    return outputs

# Use this function in your training loop:
# for images, targets in data_loader:
#     outputs = debug_forward_pass(model, images, targets)


In [ ]:
import os
import sys
import torch
from torch.utils.data import DataLoader

# Add DINO directory to system path
dino_path = "DINO"
sys.path.append(dino_path)

from util.slconfig import SLConfig
from datasets import build_dataset
from models import build_model

def debug_script():
    print("1. Loading configuration")
    config_file = 'DINO/config/DINO/DINO_4scale.py'
    args = SLConfig.fromfile(config_file)
    args.pretrain_model_path = 'DINO-4scale model/checkpoint0033_4scale.pth'
    args.output_dir = 'DINO-4scale model/DINO-4scale_Fine-tune'
    args.device = 'cuda' if torch.cuda.is_available() else 'cpu'

    print("\n2. Building dataset")
    dataset_train = build_dataset(image_set='train', args=args)
    print(f"Dataset size: {len(dataset_train)}")

    print("\n3. Checking sample item")
    sample_item = dataset_train[0]
    print(f"Sample item type: {type(sample_item)}")
    if isinstance(sample_item, tuple):
        print(f"Sample item length: {len(sample_item)}")
        print(f"Sample image shape: {sample_item[0].shape}")
        print(f"Sample target: {sample_item[1]}")

    print("\n4. Building model")
    try:
        model, criterion, postprocessors = build_model(args)
        print(f"Model structure:\n{model}")
    except Exception as e:
        print(f"Error building model: {str(e)}")
        print("Model configuration:")
        for key, value in args.items():
            print(f"{key}: {value}")

    print("\n5. Checking prepare_for_cdn function")
    utils_path = os.path.join(dino_path, "models", "dino", "utils.py")
    if os.path.exists(utils_path):
        with open(utils_path, 'r') as f:
            content = f.read()
            print("Content of utils.py:")
            print(content)
    else:
        print(f"utils.py not found at {utils_path}")

debug_script()

In [ ]:
import os
import sys
import torch
from torch.utils.data import DataLoader


dino_path = "DINO"
sys.path.append(dino_path)

from util.slconfig import SLConfig
from datasets import build_dataset
from models import build_model
from util.misc import nested_tensor_from_tensor_list

def collate_fn(batch):
    images = [item[0] for item in batch]
    targets = [item[1] for item in batch]
    batched_imgs = nested_tensor_from_tensor_list(images)
    return batched_imgs, targets

def examine_dataset(dataset, num_samples=5):
    print(f"Dataset type: {type(dataset)}")
    print(f"Dataset length: {len(dataset)}")
    for i in range(min(num_samples, len(dataset))):
        sample = dataset[i]
        print(f"\nSample {i}:")
        print(f"  Image shape: {sample[0].shape}")
        print(f"  Target keys: {sample[1].keys()}")
        print(f"  Target 'labels' shape: {sample[1]['labels'].shape}")
        print(f"  Target 'boxes' shape: {sample[1]['boxes'].shape}")

def examine_model(model):
    print(f"Model type: {type(model)}")
    print("Model parameters:")
    for name, param in model.named_parameters():
        if "prepare_for_cdn" in name:
            print(f"  {name}: {param.shape}")

def examine_config(args):
    print("Configuration parameters:")
    for key, value in args.items():
        if key in ['dn_number', 'dn_label_noise_ratio', 'dn_box_noise_scale', 'use_dn']:
            print(f"  {key}: {value}")

def main():
    # Load configuration
    config_file = 'DINO/config/DINO/DINO_4scale.py'
    args = SLConfig.fromfile(config_file)
    args.pretrain_model_path = 'DINO-4scale model/checkpoint0033_4scale.pth'
    args.output_dir = 'DINO-4scale model/DINO-4scale_Fine-tune'
    args.device = 'cuda' if torch.cuda.is_available() else 'cpu'

    print("\n1. Examining dataset:")
    dataset_train = build_dataset(image_set='train', args=args)
    examine_dataset(dataset_train)

    print("\n2. Examining dataloader:")
    data_loader_train = DataLoader(dataset_train, batch_size=2, shuffle=True, collate_fn=collate_fn, num_workers=2)
    for samples, targets in data_loader_train:
        print(f"Batch samples shape: {samples.tensors.shape}")
        print(f"Batch targets length: {len(targets)}")
        print(f"First target in batch: {targets[0].keys()}")
        break

    print("\n3. Examining model:")
    model, criterion, _ = build_model(args)
    examine_model(model)

    print("\n4. Examining configuration:")
    examine_config(args)

    print("\n5. Checking prepare_for_cdn function:")
    dn_components_path = os.path.join(dino_path, "models", "dino", "dn_components.py")
    if os.path.exists(dn_components_path):
        with open(dn_components_path, 'r') as f:
            print(f.read())
    else:
        print(f"dn_components.py not found at {dn_components_path}")

if __name__ == "__main__":
    main()

In [ ]:
import os
import sys
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
from tqdm import tqdm
from collections import defaultdict

# Add DINO directory to system path
dino_path = "DINO"
sys.path.append(dino_path)

from util.slconfig import SLConfig
from datasets import build_dataset
from models import build_dino
from util.misc import NestedTensor, nested_tensor_from_tensor_list
import models.dino.dn_components as dn_components

def collate_fn(batch):
    batch = list(zip(*batch))
    batch[0] = nested_tensor_from_tensor_list(batch[0])
    return tuple(batch)

def modified_prepare_for_cdn(dn_args, training, num_queries, num_classes, hidden_dim, label_enc):
    if training and dn_args[0] is not None:
        return dn_components.original_prepare_for_cdn(dn_args, training, num_queries, num_classes, hidden_dim, label_enc)
    return None, None, None, None

dn_components.original_prepare_for_cdn = dn_components.prepare_for_cdn
dn_components.prepare_for_cdn = modified_prepare_for_cdn

def calculate_accuracy(loss_ce):
    return max(0, 100 - loss_ce * 10)  # Scale it to a percentage

def train_one_epoch(model, criterion, data_loader, optimizer, device, epoch):
    model.train()
    criterion.train()

    metric_logger = defaultdict(float)
    num_batches = len(data_loader)

    with tqdm(total=num_batches, desc=f"Epoch {epoch+1}") as pbar:
        for samples, targets in data_loader:
            samples = samples.to(device)
            targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

            outputs = model(samples, targets)
            loss_dict = criterion(outputs, targets)
            weight_dict = criterion.weight_dict
            losses = sum(loss_dict[k] * weight_dict[k] for k in loss_dict.keys() if k in weight_dict)

            optimizer.zero_grad()
            losses.backward()
            optimizer.step()

            for k, v in loss_dict.items():
                metric_logger[k] += v.item()
            metric_logger['total_loss'] += losses.item()

            accuracy = calculate_accuracy(loss_dict['loss_ce'].item())
            metric_logger['accuracy'] += accuracy

            pbar.update(1)
            pbar.set_postfix({
                "Loss": f"{metric_logger['total_loss'] / (pbar.n + 1):.2f}",
                "Acc": f"{metric_logger['accuracy'] / (pbar.n + 1):.2f}%"
            })

    metric_logger = {k: v / num_batches for k, v in metric_logger.items()}
    return metric_logger

def main():
    config_file = 'DINO/config/DINO/DINO_4scale.py'
    args = SLConfig.fromfile(config_file)

    args.device = 'cuda' if torch.cuda.is_available() else 'cpu'
    args.lr = 1e-5
    args.lr_backbone = 1e-6
    args.batch_size = 2
    args.epochs = 100
    args.lr_drop = 4
    args.output_dir = "DINO-4scale model/DINO-4scale_Fine-tune"

    dataset_train = build_dataset(image_set='train', args=args)
    data_loader_train = DataLoader(dataset_train, batch_size=args.batch_size, shuffle=True, collate_fn=collate_fn, num_workers=2)

    model, criterion, postprocessors = build_dino(args)
    model.to(args.device)

    checkpoint = torch.load(args.pretrain_model_path, map_location=args.device)
    model.load_state_dict(checkpoint['model'], strict=False)

    param_dicts = [
        {"params": [p for n, p in model.named_parameters() if "backbone" not in n and p.requires_grad]},
        {
            "params": [p for n, p in model.named_parameters() if "backbone" in n and p.requires_grad],
            "lr": args.lr_backbone,
        },
    ]
    optimizer = optim.AdamW(param_dicts, lr=args.lr, weight_decay=args.weight_decay)
    lr_scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=args.lr_drop)

    num_epochs = args.epochs
    metrics_history = defaultdict(list)

    for epoch in range(num_epochs):
        epoch_metrics = train_one_epoch(model, criterion, data_loader_train, optimizer, args.device, epoch)
        lr_scheduler.step()

        for k, v in epoch_metrics.items():
            metrics_history[k].append(v)

        print(f"\nEpoch {epoch+1} summary:")
        print(f"  Loss: {epoch_metrics['total_loss']:.4f}")
        print(f"  Accuracy: {epoch_metrics['accuracy']:.2f}%")

        checkpoint_path = os.path.join(args.output_dir, f"checkpoint_epoch_{epoch+1}.pth")
        torch.save({
            'model': model.state_dict(),
            'optimizer': optimizer.state_dict(),
            'lr_scheduler': lr_scheduler.state_dict(),
            'epoch': epoch,
            'args': args,
        }, checkpoint_path)

    # Save final fine-tuned model
    final_model_path = os.path.join(args.output_dir, "final_model.pth")
    torch.save(model.state_dict(), final_model_path)
    print(f"\nFinal fine-tuned model saved to: {final_model_path}")

    # Plot metrics
    plt.figure(figsize=(12, 6))
    plt.plot(range(1, num_epochs+1), metrics_history['total_loss'], label='Loss')
    plt.plot(range(1, num_epochs+1), metrics_history['accuracy'], label='Accuracy')
    plt.title('Training Metrics over Epochs')
    plt.xlabel('Epoch')
    plt.ylabel('Value')
    plt.legend()
    plt.tight_layout()
    plt.savefig(os.path.join(args.output_dir, 'metrics_plot.png'))
    plt.close()

    print("Fine-tuning completed. Model and metrics saved to:", args.output_dir)

if __name__ == "__main__":
    main()

In [ ]:
import sys
import os
import torch
from tqdm import tqdm
import matplotlib.pyplot as plt
from PIL import Image
import json
import numpy as np

# Add DINO directory to sys.path
sys.path.append('DINO')

from util.slconfig import SLConfig
from models import build_model
from datasets import build_dataset
from util.misc import nested_tensor_from_tensor_list
from util.box_ops import box_cxcywh_to_xyxy
from pycocotools.coco import COCO
from pycocotools.cocoeval import COCOeval

def load_model(args):
    model, _, postprocessors = build_model(args)
    checkpoint = torch.load(args.resume, map_location='cpu')
    if isinstance(checkpoint, dict) and 'model' in checkpoint:
        model.load_state_dict(checkpoint['model'])
    else:
        model.load_state_dict(checkpoint)
    model.eval()
    return model, postprocessors

def process_image(image_path, model, postprocessors, device):
    image = Image.open(image_path).convert("RGB")
    w, h = image.size
    img = torch.from_numpy(np.array(image)).permute(2, 0, 1).float()
    img = nested_tensor_from_tensor_list([img]).to(device)

    with torch.no_grad():
        outputs = model(img)

    results = postprocessors['bbox'](outputs, torch.tensor([[w, h]]).to(device))[0]
    return results

def visualize_detections(image, boxes, labels, scores, output_path):
    plt.figure(figsize=(12, 8))
    plt.imshow(image)
    ax = plt.gca()

    for box, label, score in zip(boxes, labels, scores):
        x1, y1, x2, y2 = box
        rect = plt.Rectangle((x1, y1), x2 - x1, y2 - y1, fill=False, edgecolor='red', linewidth=2)
        ax.add_patch(rect)
        plt.text(x1, y1, f'{label}: {score:.2f}', bbox=dict(facecolor='white', alpha=0.8))

    plt.axis('off')
    plt.savefig(output_path, bbox_inches='tight', pad_inches=0)
    plt.close()

def main():
    # Load configuration
    config_file = 'DINO/config/DINO/DINO_4scale.py'
    args = SLConfig.fromfile(config_file)
    args.device = 'cuda'
    args.resume = 'DINO-4scale model/DINO-4scale_Fine-tune/final_model.pth'

    # Create output directory
    output_dir = 'output path'
    os.makedirs(output_dir, exist_ok=True)

    try:
        # Load model
        model, postprocessors = load_model(args)
        model.to(args.device)

        # Load validation dataset
        args.coco_path = 'place holder'
        args.eval = True
        dataset_val = build_dataset(image_set='val', args=args)

        # Initialize COCO API
        ann_file = os.path.join(args.coco_path, 'val', 'val.json')
        coco_gt = COCO(ann_file)

        results = []

        for img_id in tqdm(dataset_val.ids):
            # Get image path
            img_info = dataset_val.coco.loadImgs(img_id)[0]
            img_path = os.path.join(dataset_val.root, img_info['file_name'])

            # Process image
            outputs = process_image(img_path, model, postprocessors, args.device)

            # Convert to COCO format
            boxes = outputs['boxes'].tolist()
            scores = outputs['scores'].tolist()
            labels = outputs['labels'].tolist()

            for box, score, label in zip(boxes, scores, labels):
                xmin, ymin, xmax, ymax = box
                width = xmax - xmin
                height = ymax - ymin
                results.append({
                    'image_id': img_id,
                    'category_id': label,
                    'bbox': [xmin, ymin, width, height],
                    'score': score
                })

            # Visualize detections
            image = Image.open(img_path).convert("RGB")
            output_path = os.path.join(output_dir, f'detection_{img_id}.png')
            visualize_detections(image, boxes, labels, scores, output_path)

        # Save results for evaluation
        result_path = os.path.join(output_dir, 'detections.json')
        with open(result_path, 'w') as f:
            json.dump(results, f)

        # Evaluate using COCO API
        coco_dt = coco_gt.loadRes(result_path)
        coco_eval = COCOeval(coco_gt, coco_dt, 'bbox')
        coco_eval.evaluate()
        coco_eval.accumulate()
        coco_eval.summarize()

        # Save AP and AR values
        metrics_path = os.path.join(output_dir, 'metrics.txt')
        with open(metrics_path, 'w') as f:
            f.write(f"AP @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = {coco_eval.stats[0]:.3f}\n")
            f.write(f"AP @[ IoU=0.50      | area=   all | maxDets=100 ] = {coco_eval.stats[1]:.3f}\n")
            f.write(f"AP @[ IoU=0.75      | area=   all | maxDets=100 ] = {coco_eval.stats[2]:.3f}\n")
            f.write(f"AP @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = {coco_eval.stats[3]:.3f}\n")
            f.write(f"AP @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = {coco_eval.stats[4]:.3f}\n")
            f.write(f"AP @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = {coco_eval.stats[5]:.3f}\n")
            f.write(f"AR @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = {coco_eval.stats[6]:.3f}\n")
            f.write(f"AR @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = {coco_eval.stats[7]:.3f}\n")
            f.write(f"AR @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = {coco_eval.stats[8]:.3f}\n")
            f.write(f"AR @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = {coco_eval.stats[9]:.3f}\n")
            f.write(f"AR @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = {coco_eval.stats[10]:.3f}\n")
            f.write(f"AR @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = {coco_eval.stats[11]:.3f}\n")

        print(f"Evaluation complete. Results saved in {output_dir}")

    except Exception as e:
        print(f"An error occurred: {str(e)}")
        import traceback
        traceback.print_exc()

if __name__ == "__main__":
    main()